# Sensitivity Analysis

In [2]:
from model import City_Model
import numpy as np
import matplotlib.pyplot as plt
from helpers import *
from modelReporters import *
from SALib.analyze import sobol
from mesa.batchrunner import BatchRunner
from SALib.sample import saltelli
import pandas as pd

In [3]:
problem = {
    'num_vars': 4,
    'names': ['neg_coff', 'num_groups', 'radius_coff', 'N'],
    'bounds': [[0, 0.05], [3, 25], [0.125, 0.5], [1000, 3000]]
}


model_reporters={"Beta":model_beta, "Rsquare":model_rsquare, "Mean_entropy":mean_entropy}

### OFAT

In [ ]:
# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 20
max_steps = 30
distinct_samples = 15 

ofat_data = {}

for i, var in enumerate(problem['names']):
    # Get the bounds for this variable and get <distinct_samples> samples within this space (uniform)
    samples = np.linspace(*problem['bounds'][i], num = distinct_samples)
    
    # N should be integers. sample in such a way that we only get integers.
    if var == 'N' or var == 'num_groups':
        samples = np.linspace(*problem['bounds'][i], num = distinct_samples, dtype=int)
    
    batch = BatchRunner(City_Model, 
                        max_steps= max_steps,
                        iterations= replicates,
                        variable_parameters= {var: samples},
                        model_reporters= model_reporters,
                        display_progress= True)
    
    batch.run_all()
    
    ofat_data[var] = batch.get_model_vars_dataframe()

In [ ]:
def plot_param_var_conf(ax, df, var, param, i):
    """
    Helper function for plot_all_vars. Plots the individual parameter vs
    variables passed.

    Args:
        ax: the axis to plot to
        df: dataframe that holds the data to be plotted
        var: variables to be taken from the dataframe
        param: which output variable to plot
    """
    x = df.groupby(var).mean().reset_index()[var]
    y = df.groupby(var).mean()[param]

    replicates = df.groupby(var)[param].count()
    err = (1.96 * df.groupby(var)[param].std()) / np.sqrt(replicates)

    ax.plot(x, y, c='k')
    ax.fill_between(x, y - err, y + err)

    ax.set_xlabel(var)
    ax.set_ylabel(param)

def plot_all_vars(param):
    """
    Plots the parameters passed vs each of the output variables.

    Args:
        df: dataframe that holds all data
        param: the parameter to be plotted
    """

    f, axs = plt.subplots(2,2, figsize=(10, 6))
    axs = axs.flatten()
    f.suptitle("Variation in {} for different parameters".format(param),fontweight='bold')
    for i, var in enumerate(problem['names']):
        plot_param_var_conf(axs[i], ofat_data[var], var, param, i)

for param in ('Beta','Rsquare','Mean_entropy'):
    plot_all_vars(param)
    plt.tight_layout()
    plt.show()

### SOBOL

In [ ]:
# Set the repetitions, the amount of steps, and the amount of distinct values per variable
replicates = 2
max_steps = 5
distinct_samples = 2

# We get all our samples here
param_values = saltelli.sample(problem, distinct_samples)


batch = BatchRunner(City_Model, 
                    max_steps =max_steps,
                    variable_parameters={name:[] for name in problem['names']},
                    model_reporters=model_reporters)

count = 0
sob_data = pd.DataFrame(index=range(replicates*len(param_values)), 
                                columns=['neg_coff', 'num_groups', 'radius_coff', 'N'])
sob_data['Run'], sob_data['Beta'], sob_data['Rsquare'], sob_data['Mean_entropy'] = None, None, None, None

for i in range(replicates):
    for vals in param_values: 
        # Change parameters num_groups and N to integers
        vals = list(vals)
        vals[1] = int(vals[1])
        vals[3] = int(vals[3])
        
        # Transform to dict with parameter names and their values
        variable_parameters = {}
        for name, val in zip(problem['names'], vals):
            variable_parameters[name] = val

        batch.run_iteration(variable_parameters, tuple(vals), count)
        iteration_data = batch.get_model_vars_dataframe().iloc[count]
        iteration_data['Run'] = count 
        sob_data.iloc[count, 0:4] = vals
        sob_data.iloc[count, 4:8] = iteration_data

        count += 1

        clear_output()
        print(f'{count / (len(param_values) * (replicates)) * 100:.2f}% done')

In [ ]:
Si_beta = sobol.analyze(problem, sob_data['Beta'].values, print_to_console=True)
Si_rsquare = sobol.analyze(problem, sob_data['Rsquare'].values, print_to_console=True)
Si_mean_entropy = sobol.analyze(problem, sob_data['Mean_entropy'].values, print_to_console=True)

In [ ]:
def plot_index(ax, s, params, i, title=''):
    """
    Creates a plot for Sobol sensitivity analysis that shows the contributions
    of each parameter to the global sensitivity.

    Args:
        ax: an axes to plot upon. 
        s (dict): dictionary {'S#': dict, 'S#_conf': dict} of dicts that hold
            the values for a set of parameters
        params (list): the parameters taken from s
        i (str): string that indicates what order the sensitivity is.
        title (str): title for the plot
    """

    if i == '2':
        p = len(params)
        params = list(combinations(params, 2))
        indices = s['S' + i].reshape((p ** 2))
        indices = indices[~np.isnan(indices)]
        errors = s['S' + i + '_conf'].reshape((p ** 2))
        errors = errors[~np.isnan(errors)]
    else:
        indices = s['S' + i]
        errors = s['S' + i + '_conf']
        
    l = len(indices)

    ax.set_title(title, fontsize=12)
    ax.set_ylim([-0.2, len(indices) - 1 + 0.2])
    ax.set_yticks(range(l))
    ax.set_yticklabels(params, fontsize=9, rotation=30) # labels
    ax.errorbar(indices, range(l), xerr=errors, linestyle='None', marker='o')
    ax.axvline(0, c='k')

In [ ]:
fig, axs = plt.subplots(nrows=3,ncols=2,figsize=(12,9),dpi=200)

axs = axs.flatten()
row_labels = [r'$\beta$',r'$R^2$','Entropy']
for i, Si in enumerate([Si_beta, Si_rsquare, Si_mean_entropy]):
    # First order
    plot_index(axs[2*i], Si, problem['names'], '1', 'First order sensitivity')
    axs[2*i].set_ylabel(row_labels[i], rotation=0, size=15)
    axs[2*i].yaxis.labelpad = 25
    

    '''
    # Second order
    plot_index(axs[3*i+1], Si, problem['names'], '2', 'Second order sensitivity')
    #plt.show()'''

    # Total order
    plot_index(axs[2*i+1], Si, problem['names'], 'T', 'Total order sensitivity')
    
    
plt.tight_layout() 